# Build a Fully Local RAG Application With PostgreSQL, Mistral, and Ollama

## Install & import libraries

The required library `psycopg2` is imported to handle PostgreSQL database operations.


In [13]:
# %pip install psycopg2-binary

In [ ]:
import os
import psycopg2

# os.environ["OLLAMA_HOST"] = "localhost"
# os.environ["OLLAMA_ORIGINS"] = "http://localhost:11434"

## Dataset Preparation

In this tutorial, we will create dummy data of different locations with their description. 

In [15]:
# the knowledge base
dummy_data = [
    {"title": "Seoul Tower", "content": "Seoul Tower is a communication and observation tower located on Namsan Mountain in central Seoul, South Korea."},
    {"title": "Gwanghwamun Gate", "content": "Gwanghwamun is the main and largest gate of Gyeongbokgung Palace, in Jongno-gu, Seoul, South Korea."},
    {"title": "Bukchon Hanok Village", "content": "Bukchon Hanok Village is a Korean traditional village in Seoul with a long history."},
    {"title": "Myeong-dong Shopping Street", "content": "Myeong-dong is one of the primary shopping districts in Seoul, South Korea."},
    {"title": "Dongdaemun Design Plaza", "content": "The Dongdaemun Design Plaza is a major urban development landmark in Seoul, South Korea."}
]

## Database connection component

In [16]:
def connect_db():
    return psycopg2.connect(
        host = 'localhost',
        database = 'postgres',
        user = 'postgres',
        password = 'postgres',
        port = '5432'
    )

## Table creation component
This component creates the documents table if it does not already exist. The table includes columns for `id`, `title`, `content`, and `embedding`. 

In [17]:
with connect_db() as conn:
    with conn.cursor() as cur:
        cur.execute("""
            CREATE TABLE IF NOT EXISTS documents (
                id SERIAL PRIMARY KEY,
                title TEXT,
                content TEXT,
                embedding VECTOR(128)
            );
        """)

## Data insertion component
This component inserts data into the documents table. It first inserts the title and content then updates the embedding field using the [`ollama_embed`](https://github.com/timescale/pgai/blob/main/docs/ollama.md#embed) function from pgai.


In [18]:
with connect_db() as conn:
    with conn.cursor() as cur:
        for doc in dummy_data:
            cur.execute(
                """
                INSERT INTO documents (title, content, embedding)
                VALUES (
                    %(title)s,
                    %(content)s,
                    ai.ollama_embed(
                        'bert_tiny_embeddings_english_portuguese-unsloth',
                        concat(%(title)s, ' - ', %(content)s),
                        host=>'http://localhost:11434'
                    )
                );
            """,
                doc,
            )

Safechecking the insert information in the database.

In [ ]:
with connect_db() as conn:
    with conn.cursor() as cur:
        cur.execute("""
            SELECT title, content, embedding
            FROM documents;
        """)

        rows = cur.fetchall()
        for row in rows:
            print(f"Title: {row[0]}\nContent: {row[1]}\nEmbedding: {row[2][0:150]}...\n\n")

Title: Seoul Tower
Content: Seoul Tower is a communication and observation tower located on Namsan Mountain in central Seoul, South Korea.
Embedding: [-0.49723864,0.49548247,-0.97343874,1.1164243,1.5570251,-1.7529635,0.36347806,0.8493127,-0.41763458,-0.15788816,-0.20899925,-0.14316028,-0.6062061,-0....


Title: Gwanghwamun Gate
Content: Gwanghwamun is the main and largest gate of Gyeongbokgung Palace, in Jongno-gu, Seoul, South Korea.
Embedding: [-0.96670383,0.37753496,-1.2280794,0.73322725,2.0414915,-2.3392425,0.45784143,1.1790159,0.3963067,0.13546674,0.54765046,0.31239516,-1.4799881,-0.35045...


Title: Bukchon Hanok Village
Content: Bukchon Hanok Village is a Korean traditional village in Seoul with a long history.
Embedding: [-1.0541975,-0.08922814,-0.5264848,1.0680342,1.5769306,-2.8867817,0.38844135,1.0687172,0.005215177,-0.052269667,-0.533085,0.4937006,-1.3632596,0.49114...


Title: Myeong-dong Shopping Street
Content: Myeong-dong is one of the primary shopping districts in Seoul

## Retrieve and generate response component

This component takes a query, embeds it, retrieves the most relevant documents based on cosine similarity, and generates a response using the [`ollama_generate`](https://github.com/timescale/pgai/blob/main/docs/ollama.md#generate) function.

In [20]:
query = "Tell me about landmarks in Seoul"

In [23]:
with connect_db() as conn:
    with conn.cursor() as cur:
        # Embed the query using the ollama_embed function
        cur.execute(
            """
            SELECT ai.ollama_embed(
                    'bert_tiny_embeddings_english_portuguese-unsloth',
                    %s,
                    host=>'http://localhost:11434'
            );
        """,
            (query,),
        )
        query_embedding = cur.fetchone()[0]

        # Retrieve relevant documents based on cosine distance
        cur.execute("""
            SELECT title, content, 1 - (embedding <=> %s) AS similarity
            FROM documents
            ORDER BY similarity DESC
            LIMIT 3;
        """, (query_embedding,))

        rows = cur.fetchall()

        # Prepare the context for generating the response
        context = "\n\n".join([f"Landmark: {row[0]}\nDescription: {row[1]}" for row in rows])

        # Generate the response using the ollama_generate function
        cur.execute(
            """
            SELECT ai.ollama_generate(
                    'mistral_7b_portuguese-unsloth', %s,
                    host=>'http://localhost:11434'
            );
        """,
            (f"Query: {query}\nContext: {context}",),
        )

        model_response = cur.fetchone()[0]
        print(model_response['response'])

Acho o Seoul Tower para ser um grande marco em Seul. É uma torre de comunicação e observação que oferece vistas panorâmicas da cidade. O Dongdaemun Design Plaza é outro marco emblemático na cidade, conhecido por sua arquitetura única e eventos de arte moderna. Finalmente, o Myeong-dong Shopping Street é uma ótima área para ir se você quiser explorar a cultura e a vida diária da Seul.
